In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.utils import Sequence
import matplotlib.pyplot as plt

In [2]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Function to read annotation data from the .cat files
def read_annotation_file(annotation_file_path):
    with open(annotation_file_path, 'r') as file:
        line = file.readline().strip()
        values = line.split()
        num_points = int(values[0])
        annotation_data = [int(value) for value in values[1:]]
        return num_points, annotation_data

# Path to the directory containing the images and .cat files
folder_path = r'C:\Users\haris\AI\ML\Projects\Data\CatFaceFeatures\Cats - Original'

# Lists to store paths of .jpg files and .cat files
image_files_path = []
annotation_files_path = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', 'png')):
            image_path = os.path.join(root, file)
            image_files_path.append(image_path)

        if file.lower().endswith('.cat'):
            annotation_path = os.path.join(root, file)
            annotation_files_path.append(annotation_path)

# Lists to store resized image data and annotation data
resized_images = []
resized_annotations = []

image_size = (224, 224)

# Read annotation data for each image and apply normalization
for annotation_file_path in annotation_files_path:
    num_points, annotation_data = read_annotation_file(annotation_file_path)

    image = cv2.imread(annotation_file_path[:-4], cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image, image_size)

    # Calculate the scaling factor for annotation coordinates
    scale_y = resized_image.shape[0] / image.shape[0]
    scale_x = resized_image.shape[1] / image.shape[1]

    # Apply scaling to the annotation data for both x and y coordinates together
    resized_annotation_data = [int(value * scale_x) if i % 2 == 0 else int(value * scale_y) for i, value in enumerate(annotation_data)]

    resized_images.append(resized_image)
    resized_annotations.append(resized_annotation_data)

# Convert lists to numpy arrays
resized_images = np.array(resized_images)
resized_annotations = np.array(resized_annotations)


In [3]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(resized_images, resized_annotations, test_size=0.1, random_state=42)

# Function to load and preprocess the images and annotations in batches
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array(batch_x), np.array(batch_y)

In [6]:
# Define the model with regularization
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = BatchNormalization()(x)
predictions = Dense(18, activation='linear')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze layers starting from a certain index (e.g., unfreeze layers from 100 onwards)
for layer in model.layers[100:]:
    layer.trainable = True

# Compile the model
initial_learning_rate = 0.0001
optimizer = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with data generator
batch_size = 8
train_gen = DataGenerator(x_train, y_train, batch_size)
test_gen = DataGenerator(x_test, y_test, batch_size)

# Train the model with data generator
history = model.fit(
    train_gen,
    epochs=15,
    validation_data=test_gen,
    callbacks=[early_stopping]
)
# Function to plot the training history
def plot_training_history(history):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Training MAE')
    plt.plot(history.history['val_mae'], label='Validation MAE')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Absolute Error (MAE)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Plot the training history
plot_training_history(history)


Epoch 1/8
1125/1125 [==============================] - 153s 129ms/step - loss: 10091.6211 - mae: 91.6281 - val_loss: 9618.8262 - val_mae: 90.3144
Epoch 2/8
1125/1125 [==============================] - 136s 121ms/step - loss: 9066.6523 - mae: 87.5026 - val_loss: 7880.3149 - val_mae: 81.5333
Epoch 3/8
1125/1125 [==============================] - 147s 130ms/step - loss: 7422.4194 - mae: 78.7177 - val_loss: 6500.7998 - val_mae: 73.6468
Epoch 4/8
1125/1125 [==============================] - 166s 147ms/step - loss: 5463.3628 - mae: 66.2166 - val_loss: 4588.1348 - val_mae: 60.3210
Epoch 5/8
1125/1125 [==============================] - 143s 127ms/step - loss: 3592.6882 - mae: 51.7386 - val_loss: 2751.9417 - val_mae: 44.2943
Epoch 6/8
1125/1125 [==============================] - 140s 125ms/step - loss: 2088.0750 - mae: 37.1669 - val_loss: 1309.9443 - val_mae: 27.7875
Epoch 7/8
1125/1125 [==============================] - 150s 134ms/step - loss: 1101.5157 - mae: 25.2647 - val_loss: 643.6977 - va

In [ ]:
model.summary()

In [10]:
# Evaluate the model on the test set
evaluation = model.evaluate(test_gen, verbose=0)
print("Test Loss:", evaluation[0])
print("Test MAE:", evaluation[1])

# Make predictions using the trained model
predictions = model.predict(test_gen)

# Calculate and print some example predictions and actual values
for i in range(5):
    print("Example", i+1)
    print("Predicted:", predictions[i])
    print("Actual:", y_test[i])
    print("")

# Calculate the mean absolute error (MAE) of the predictions
mae = np.mean(np.abs(predictions - y_test))
print("Mean Absolute Error (MAE):", mae)


Test Loss: 322.2524108886719
Test MAE: 12.117645263671875
125/125 [==============================] - 3s 23ms/step
Example 1
Predicted: [133.43936  104.63026  150.70647  116.362434 135.73407  128.2163
 120.09697   78.546005 125.88072   32.86334  139.28737   69.43165
 152.45874   82.1323   162.68492   69.45744  153.4139   107.99866 ]
Actual: [135 103 160 114 132 135 133  83 133  38 148  73 175  83 206  62 196 112]

Example 2
Predicted: [107.56317   49.60391  120.7434    51.21089  112.84858   66.11668
  94.52711   37.583775  91.001366  10.084259 106.75842   24.801477
 116.54228   25.536345 123.237625   9.893718 117.179405  42.12436 ]
Actual: [127  42 154  43 150  58  87  35  89   6 116  23 144  25 166  14 162  40]

Example 3
Predicted: [ 97.02936  122.32153  129.20622  139.51907  104.99069  152.28407
  74.083565  85.85626   85.89133   17.36649  108.263435  72.05338
 136.86436   92.923546 154.3864    77.44414  141.52005  128.74565 ]
Actual: [ 83 129 137 163  91 191  65  80  93   9 114  75 